In [ ]:
# cd ~/twinews-logs ; jupython --venv st-venv ~/notebooks/twinews/dump.ipynb

In [ ]:
isNotebook = '__file__' not in locals()

In [ ]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from databasetools.mongo import *
from newstools.goodarticle.utils import *
from nlptools.preprocessing import *
from nlptools.news import parser as newsParser
from machinelearning.iterator import *
from twinews.utils import *

In [ ]:
logger = Logger(tmpDir('logs') + "/twinews-dump.log") if isNotebook else Logger("dump.log")
tt = TicToc(logger=logger)
tt.tic()

In [ ]:
newsCollection = getNewsCollection(logger=logger, user=hjuser, password=hjpass, host=hjhost)
usersCollection = getUsersCollection(logger=logger, user=hjuser, password=hjpass, host=hjhost)

In [ ]:
outputDir = nosaveDir() + "/twinews-dumps/" + getDateSec()

In [ ]:
def ndjsonDump(gen, outputDir, maxMoSize=20, refreshEach=10, size=None,
               printRatio=0.01, logger=None, verbose=True):
    mkdir(outputDir)
    i = -1
    currentFile = None
    pbar = None
    if size is None:
        try:
            size = len(gen)
        except: pass
    if size is not None:
        pbar = ProgressBar(size, logger=logger, verbose=verbose, printRatio=printRatio)
    for row in gen:
        if currentFile is None or currentFile.getEstimatedMoSize(refreshEach=refreshEach) > maxMoSize:
            if currentFile is not None:
                currentFile.close()
            i += 1
            currentPath = outputDir + "/" + str(i) + ".ndjson.bz2"
            assert not isFile(currentPath)
            currentFile = NDJson(currentPath, closeAtEachAppend=False)
        currentFile.append(row)
        if pbar is not None:
            pbar.tic()
    if currentFile is not None:
        currentFile.close()

In [ ]:
def gen(collection):
    for row in collection.find():
        del row['_id']
        yield row

In [ ]:
ndjsonDump(gen(newsCollection), outputDir + "/news", size=len(newsCollection), refreshEach=100, logger=logger)

In [ ]:
ndjsonDump(gen(usersCollection), outputDir + "/users", size=len(usersCollection), refreshEach=3, logger=logger)